# Performance

Obviously, using unit-aware variables will slow down any computation compared to raw python values (int, flot, numpy.ndarray).

In [6]:
import matplotlib.pyplot as plt
import numpy as np

import physipy
from physipy import s, m, setup_matplotlib

from physipy import Dimension, units, quantify, Quantity



ms = units["ms"]
mm = units['mm']
km = units["km"]
cm = units["cm"]
mus = units["mus"]
ns = units["ns"]
a = 123456
b = 654321

aq = a*m
bq = b*m


Basic comparison on addition

In [10]:
%timeit  (a +  b)
%timeit (aq + bq)

63.7 ns ± 2.37 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
12.4 µs ± 446 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
print(12.4*mus/(63.7*ns))

194.66248037676607


Basic comparison on pow

In [12]:
%timeit  (a**2)
%timeit (aq**2)

289 ns ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
22.8 µs ± 989 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
print(22.8*mus/(289*ns))

78.89273356401384


## benchmark timing

Here is a comparison of most operations : 

In [32]:
import timeit

operations = {
    "add":"__add__", 
    #"sub":"__sub__",
    #"mul":"__mul__",
}

a = 123456
b = 654321

aq = a*m
bq = b*m

for operation, operation_method in operations.items():
    time = timeit.timeit('a.'+operation_method+"(b)", number=10000, globals=globals())
    time_q = timeit.timeit('aq.'+operation_method+"(bq)", number=10000, globals=globals())
    print(f"{m: >5} : {time_q/time: <5.1f}")

  add : 1802.0


# Dimension performance

Most important operators : 
 - equlity check
 - Addition of values key-wise
 - Substration of values key-wise
 - Multiply values
 - Div
 - Pow
 

In [13]:
import numpy as np
import collections
"""Goal : return True if 2 vectors of numbers are equal
Inputs :
 - vectors are assured to be the same size
 - vector values can be int, float, np.numbers, fractions
 - the order of the numbers matters (like with dict comparison or ordered dict)
"""
 
as_dictl = {"A":0, "B":0, "C":0}
as_dictr = {"A":0, "B":0, "C":0}
as_listl = [0, 0, 0]
as_listr = [0, 0, 0]
as_arryl = np.array([0, 0, 0])
as_arryr = np.array([0, 0, 0])
as_odictl = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_odictr = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_counterl = collections.Counter("AAABBBCCC")
as_counterr = collections.Counter("AAABBBCCC")

In [14]:
%timeit as_listl == as_listr
%timeit as_dictl == as_dictr
%timeit as_counterl == as_counterr
%timeit as_odictl == as_odictr
%timeit as_arryl.tolist() == as_arryr.tolist()
%timeit list(as_odictl.values()) == list(as_odictr.values())
%timeit np.array_equal(as_arryl, as_arryr)
%timeit np.all(as_arryl == as_arryr)


47.2 ns ± 2.02 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
77.4 ns ± 1.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
79.2 ns ± 0.916 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
86.9 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
324 ns ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
799 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
5.22 µs ± 572 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.35 µs ± 409 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
a = np.arange(500)
b = np.arange(500)

%timeit np.all(a == b)
%timeit a.tolist() == b.tolist()


5 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
17.7 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
import numpy as np
import collections
from operator import add


as_dictl = {"A":0, "B":0, "C":0}
as_dictr = {"A":0, "B":0, "C":0}
as_listl = [0, 0, 0]
as_listr = [0, 0, 0]
as_arryl = np.array([0, 0, 0])
as_arryr = np.array([0, 0, 0])
as_odictl = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_odictr = collections.OrderedDict( {"A":0, "B":0, "C":0})

%timeit [l+r for l,r in zip(as_listl, as_listr)]
%timeit {k:as_dictl[k]+as_dictr[k] for k in (as_dictl.keys() & as_dictr.keys())}
#%timeit as_odictl == as_odictr
#%timeit as_arryl.tolist() == as_arryr.tolist()
#%timeit list(as_odictl.values()) == list(as_odictr.values())
#%timeit np.array_equal(as_arryl, as_arryr)
%timeit as_arryl + as_arryr
%timeit list(map(add, as_listl, as_listr))

616 ns ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.35 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
600 ns ± 45.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
624 ns ± 61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
import numpy as np
import collections
from operator import mul

as_dictl = {"A":0, "B":0, "C":0}
as_dictr = {"A":0, "B":0, "C":0}
as_listl = [0, 0, 0]
as_listr = [0, 0, 0]
as_arryl = np.array([0, 0, 0])
as_arryr = np.array([0, 0, 0])
as_odictl = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_odictr = collections.OrderedDict( {"A":0, "B":0, "C":0})

%timeit [l*r for l,r in zip(as_listl, as_listr)]
%timeit {k:as_dictl[k]*as_dictr[k] for k in (as_dictl.keys() & as_dictr.keys())}
#%timeit as_odictl == as_odictr
#%timeit as_arryl.tolist() == as_arryr.tolist()
#%timeit list(as_odictl.values()) == list(as_odictr.values())
#%timeit np.array_equal(as_arryl, as_arryr)
%timeit as_arryl * as_arryr
%timeit list(map(mul, as_listl, as_listr))

685 ns ± 145 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.15 µs ± 87.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
544 ns ± 50.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
574 ns ± 36.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [2]:
import numpy as np
import collections
from operator import pow

as_dictl = {"A":1, "B":1, "C":1}
as_dictr = 2
as_listl = [1, 1, 1]
as_listr = 2
as_arryl = np.array([1, 1, 1])
as_arryr = 2
as_odictl = collections.OrderedDict( {"A":1, "B":1, "C":1})
as_odictr = 2

%timeit [l**as_dictr for l in as_listl]
%timeit {k:as_dictl[k]**as_dictr for k in as_dictl.keys()}
%timeit as_arryl ** as_arryr
%timeit list(map(lambda x:x**2, as_listl))

980 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.19 µs ± 12.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
733 ns ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.27 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Array creation

Compare lazy creation of arrays

In [ ]:
%timeit asqarray([0*m, 2*m])
%timeit [0, 2]*m